In [27]:
# Import modules

import pandas as pd
import numpy as np
import re
import chardet

In [28]:
# Some constants

us_state_to_code = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'District of Columbia': 'DC',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    'United States': 'US'
}

# Reverse dictionary for lookup in the opposite direction
us_code_to_state = {v: k for k, v in us_state_to_code.items()}

In [29]:
# Part-1: Clean US 2024 Presidential Election Results (state level)
# Load the Data
with open('Datasets/2024-results-statelevel.csv', 'rb') as f:
    result = chardet.detect(f.read())
    print(result)
df2024_president_state = pd.read_csv('Datasets/2024-results-statelevel.csv', encoding=result['encoding'])
df2024_president_state = df2024_president_state.head(52)
print(df2024_president_state)

# Rename certain attributes
df2024_president_state = df2024_president_state.rename(columns={
    'STATE': 'state_code',
    'ELECTORAL VOTES': 'evs',
    'ELECTORAL VOTE: TRUMP (R)': 'r_evs',
    'ELECTORAL VOTE: HARRIS (D)': 'd_evs',
    'HARRIS': 'total_d_votes',
    'TRUMP': 'total_r_votes',
    'TOTAL VOTES': 'total_votes'
})

# List of columns to clean
columns_to_clean = ['total_d_votes', 'total_r_votes', 'total_votes']

# Remove commas and convert to integers
for col in columns_to_clean:
    df2024_president_state[col] = df2024_president_state[col].str.replace(',', '').astype(int)

# Calculate the Other columns
df2024_president_state['total_others_votes'] = df2024_president_state['total_votes'] - df2024_president_state['total_r_votes'] - df2024_president_state['total_d_votes']
df2024_president_state['others_vote_percentage'] = (df2024_president_state['total_others_votes'] / df2024_president_state['total_votes']) * 100
df2024_president_state['others_vote_percentage'] = df2024_president_state['others_vote_percentage'].round(2)

# Calculate the vote percentages
df2024_president_state['d_vote_percentage'] = (df2024_president_state['total_d_votes'] / df2024_president_state['total_votes']) * 100
df2024_president_state['d_vote_percentage'] = df2024_president_state['d_vote_percentage'].round(2)
df2024_president_state['r_vote_percentage'] = (df2024_president_state['total_r_votes'] / df2024_president_state['total_votes']) * 100
df2024_president_state['r_vote_percentage'] = df2024_president_state['r_vote_percentage'].round(2)

# Add the state name for each state_code
df2024_president_state['state_name'] = df2024_president_state['state_code'].map(us_code_to_state)

# Add other columns
df2024_president_state['year'] = 2024
df2024_president_state['d_candidate'] = "Kamala Harris"
df2024_president_state['r_candidate'] = "Donald Trump"
df2024_president_state['margin'] = df2024_president_state['d_vote_percentage'] - df2024_president_state['r_vote_percentage']

# Create the final data frame
attributes = [
    "year",
    "state_name",
    "state_code",
    "d_candidate",
    "total_d_votes",
    "d_vote_percentage",
    "d_evs",
    "r_candidate",
    "total_r_votes",
    "r_vote_percentage",
    "r_evs",
    "total_others_votes",
    "others_vote_percentage",
    "total_votes",
    "margin"
]

df2024_president_state = df2024_president_state[attributes]

# Final Touches
df2024_president_state.fillna(0, inplace=True)
df2024_president_state.at[51, "state_name"] = "United States"
df2024_president_state.at[51, "state_code"] = "US"
df2024_president_state["evs"] = df2024_president_state["d_evs"] + df2024_president_state["r_evs"]
df2024_president_state.rename(columns={"margin": "diff_percent"}, inplace=True)
df2024_president_state['diff_percent'] = -df2024_president_state['diff_percent']

# Export the data
df2024_president_state.to_csv('FinalDatasets/2024-president-statelevel-results-final.csv', index=False, encoding='utf-8-sig')

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}
     STATE ELECTORAL VOTES  ELECTORAL VOTE: TRUMP (R)  \
0       AL               9                        9.0   
1       AK               3                        3.0   
2       AZ              11                       11.0   
3       AR               6                        6.0   
4       CA              54                        NaN   
5       CO              10                        NaN   
6       CT               7                        NaN   
7       DE               3                        NaN   
8       DC               3                        NaN   
9       FL              30                       30.0   
10      GA              16                       16.0   
11      HI               4                        NaN   
12      ID               4                        4.0   
13      IL              19                        NaN   
14      IN              11                       11.0   
15      IA             

In [30]:
# Part-2: Clean US 2024 Senate Election Results
# Step 1: Load and filter
df2024_senate_state = pd.read_csv('Datasets/2024-senate-results.csv', encoding=result['encoding'])
df2024_senate_state = df2024_senate_state[
    (df2024_senate_state['cycle'] == 2024) & 
    (df2024_senate_state['percent'] >= 5)
][['state', 'state_abbrev', 'office_seat_name', 'special', 'candidate_name', 'ballot_party', 'votes', 'percent']]

# Step 2: Remove CA special
df2024_senate_state = df2024_senate_state[
    ~((df2024_senate_state['state_abbrev'] == 'CA') & (df2024_senate_state['special'] == 'TRUE'))
]

# Step 3: Fix Dan Osborn Party
df2024_senate_state.loc[df2024_senate_state['candidate_name'] == 'Dan Osborn', 'ballot_party'] = 'IND'

# Step 4: Sort by race and votes descending
group_cols = ['state', 'state_abbrev', 'office_seat_name', 'special']
df2024_senate_state = df2024_senate_state.sort_values(group_cols + ['votes'], ascending=[True, True, True, True, False])

# Step 5: Assign rank and mark 'Others'
df2024_senate_state['cand_num'] = df2024_senate_state.groupby(group_cols).cumcount() + 1
df2024_senate_state['is_other'] = df2024_senate_state['cand_num'] > 2

# Step 6: Assign top 2 candidates per race
df2024_senate_state['cand_num'] = df2024_senate_state.groupby(group_cols)['votes'].rank(method='first', ascending=False)
top2 = df2024_senate_state[df2024_senate_state['cand_num'] <= 2].copy()

# Step 7: Pivot top 2 to columns
pivoted = top2.pivot_table(
    index=group_cols,
    columns='cand_num',
    values=['candidate_name', 'ballot_party', 'votes', 'percent'],
    aggfunc='first'
)

# Step 8: Flatten columns
pivoted.columns = [f'{col[0]}_{int(col[1])}' for col in pivoted.columns]
pivoted = pivoted.reset_index()

# Step 9: Calculate "Others"
pivoted['others_percent'] = 100 - (pivoted['percent_1'] + pivoted['percent_2'])
pivoted['others_votes'] = (pivoted['others_percent']/pivoted['percent_1'])*pivoted['votes_1']
pivoted['total_votes'] = pivoted['others_votes'] + pivoted['votes_1'] + pivoted['votes_2']

# Optional: clean up
pivoted = pivoted.sort_values(by='state')
pivoted.fillna(0, inplace=True)

# Add a row that aggregates count for all rows to get total senate votes in the U.S. by adding the total votes of all rows
col_sums = pivoted[['votes_1', 'votes_2', 'others_votes', 'total_votes']].sum()
summary_row = pd.DataFrame([{
    'state':            'United States',
    'state_abbrev':     'US',
    'office_seat_name': '',
    'special':          False,
    'candidate_name_1': '',
    'ballot_party_1':   '',
    'votes_1':          col_sums['votes_1'],
    'percent_1':        0,
    'candidate_name_2': '',
    'ballot_party_2':   '',
    'votes_2':          col_sums['votes_2'],
    'percent_2':        0,
    'others_percent':   0,
    'others_votes':     col_sums['others_votes'],
    'total_votes':      col_sums['total_votes'],
}], columns=pivoted.columns)
pivoted = pd.concat([pivoted, summary_row], ignore_index=True)

# Load the turnout data at the state level Data
df2024_turnout_president = pd.read_csv('Datasets/2024-turnout-president.csv', encoding=result['encoding'])
df2024_turnout_president = df2024_turnout_president.rename(columns={'STATE': 'state_name', 'STATE_ABV': 'state_code'})

# Remove commas and convert to integers
df2024_turnout_president['VEP'] = df2024_turnout_president['VEP'].str.replace(',', '').astype(int)
turnout_subset = df2024_turnout_president[['state_code', 'VEP']]

# left-merge into pivoted
pivoted = pivoted.rename(columns={'state': 'state_name', 'state_abbrev': 'state_code'})
pivoted = pivoted.merge(
    turnout_subset,
    on='state_code',
    how='left'
)

pivoted['senate_turnout'] = ((pivoted['total_votes'] / pivoted['VEP'])*100).round(2)
pivoted['margin'] = pivoted['percent_1'] - pivoted['percent_2']

for idx, row in pivoted.iterrows():
    if row['state_code'] == 'US':
        continue
    if row['ballot_party_1'] != 'REP':
        pivoted.at[idx, 'margin'] = -row['margin']

# Save
pivoted.to_csv('FinalDatasets/2024-senate-results-final.csv', index=False, encoding='utf-8-sig')

In [31]:
# Part-3: Clean US 2024 House  Election Results
df2024_house_results = pd.read_csv('Datasets/ushouse2024-results.csv', encoding=result['encoding'])

# Only fill the string columns with 'NA'
string_cols = ['Candidate 2','Party 2','Winner 2','Incumbent 2']
df2024_house_results[string_cols] = df2024_house_results[string_cols].fillna('NA')

for c in ['Votes 1','Votes 2','Percent 2']:
    df2024_house_results[c] = (
        pd.to_numeric(df2024_house_results[c], errors='coerce')  # force into numbers, NaN if bogus
          .fillna(0)
          .astype(float)
    )

df2024_house_results['total_votes'] = df2024_house_results["Votes 1"] + np.where(df2024_house_results["Votes 2"] != "NA", df2024_house_results["Votes 2"].astype(int), 0)
df2024_house_results.columns = df2024_house_results.columns.str.lower().str.replace(' ', '_')
df2024_house_results['margin'] = df2024_house_results['percent_1'] - df2024_house_results['percent_2']

for idx, row in df2024_house_results.iterrows():
    if row['party_1'] != 'R':
        df2024_house_results.at[idx, 'margin'] = -row['margin']

In [32]:
# Part-4: Clean US 2024 Presidential Election Results at the County level
# Load the Data
df2024_president_county = pd.read_csv('Datasets/2024-results-countylevel.csv', encoding=result['encoding'])

columns_to_scale = ['per_gop', 'per_dem', 'per_point_diff']
df2024_president_county[columns_to_scale] = df2024_president_county[columns_to_scale].apply(lambda x: (x * 100).round(2))
df2024_president_county["state_code"] = df2024_president_county["state_name"].map(us_state_to_code)

df2024_president_county.to_csv('FinalDatasets/2024-president-countylevel-results-final.csv', index=False, encoding='utf-8-sig')

In [33]:
# Part-5: Turnout data of congressional districts
# Load the Data
df2024_cd_turnout = pd.read_csv('Datasets/cd-population.csv', encoding=result['encoding'])

# Extract relevant columns
df2024_cd_turnout = df2024_cd_turnout[["geoname", "lntitle", "tot_est", "adu_est", "cit_est", "cvap_est"]].copy()
df2024_cd_turnout = df2024_cd_turnout[~df2024_cd_turnout["geoname"].str.contains("Delegate District", case=False, na=False)]
df2024_cd_turnout = df2024_cd_turnout[~df2024_cd_turnout["geoname"].str.contains("Resident Commissioner", case=False, na=False)]
df2024_cd_turnout = df2024_cd_turnout[~df2024_cd_turnout["geoname"].str.contains("not defined", case=False, na=False)]

# Function to handle both standard and at-large districts
def convert_geoname_to_code(geoname):
    # If it's an at Large district
    if "at Large" in geoname:
        state = geoname.split(",")[-1].strip()
        state_abbr = us_state_to_code.get(state, "NA")
        return f"{state_abbr} 01"
    
    # Otherwise handle regular district
    match = re.match(r"Congressional District (\d+).*?,\s*(.+)", geoname)
    if match:
        district, state = match.groups()
        state_abbr = us_state_to_code.get(state.strip(), "NA")
        return f"{state_abbr} {district.zfill(2)}"
    
    return geoname

df2024_cd_turnout["geoname"] = df2024_cd_turnout["geoname"].apply(convert_geoname_to_code)
df2024_cd_turnout.fillna(0)
df2024_cd_turnout.rename(columns={'geoname': 'district'}, inplace=True)
df2024_cd_turnout = df2024_cd_turnout[df2024_cd_turnout['lntitle'] == 'Total']

total_house_votes = df2024_house_results[["district", "total_votes"]]

# Add total vote details to the df
df2024_cd_turnout = df2024_cd_turnout.merge(
    total_house_votes,
    on="district",
    how="left"
)

# Replace any missing Total Votes with 0, just in case
df2024_cd_turnout["total_votes"].fillna(0)

# Calculate vap_turnout_rate
df2024_cd_turnout["vap_turnout_rate"] = (
    (df2024_cd_turnout["total_votes"] / df2024_cd_turnout["cvap_est"]) * 100
)

df2024_cd_turnout = df2024_cd_turnout[['district', 'total_votes', 'cvap_est', 'vap_turnout_rate']]

# Add turnout rate to house results
merged = pd.merge(
    df2024_house_results,
    df2024_cd_turnout[['district', 'vap_turnout_rate']],
    on=['district'],
    how='outer',
    indicator=True
)

df2024_house_results = merged.loc[merged['_merge'] == 'both', 
    [col for col in merged.columns if col != '_merge']
].reset_index(drop=True)
df2024_house_results.rename(columns={'vap_turnout_rate': 'house_turnout'}, inplace=True)

df2024_house_results.to_csv('FinalDatasets/2024-house-results-final.csv', index=False, encoding='utf-8-sig')
df2024_cd_turnout.to_csv('FinalDatasets/2024-house-turnout-final.csv', index=False, encoding='utf-8-sig')

In [34]:
# Part-5: Turnout data of states
merged_turnout = pd.merge(
    df2024_turnout_president,
    pivoted,
    on=['state_name', 'state_code'],
    how='outer',        # keep everything so we can spot non-matches
    indicator=True      # adds a "_merge" column
)

df2024_turnout_president = df2024_turnout_president[['state_name', 'state_code', 'TOTAL_BALLOTS_COUNTED', 'VAP', 'VEP', 'VEP_TURNOUT_RATE', 'VAP_TURNOUT_RATE']]
df2024_turnout_president.columns = df2024_turnout_president.columns.str.lower().str.replace(' ', '_')
df2024_turnout_president.to_csv('FinalDatasets/2024-president-turnout-final.csv', index=False, encoding='utf-8-sig')

In [35]:
# Part-6: Extract Race, Age, and Sex details from a county
# Load the Data
# df1 = pd.read_csv('Datasets/2020-2023-RaceAgeSex-AllCounties.csv', encoding=result['encoding'])
df2 = pd.read_csv('Datasets/cc-est2023-agesex-all.csv', encoding=result['encoding'])

# Only consider the latest details
df2 = df2[df2["YEAR"] == 5]

# Initialize County Details dataframe
df_county_details = pd.DataFrame()
df2["state_code"] = df2["STNAME"].map(us_state_to_code)
df_county_details['state_name'] = df2['STNAME']
df_county_details['state_code'] = df2['state_code']
df_county_details['county_name'] = df2['CTYNAME']
df_county_details['population'] = df2['POPESTIMATE']

# Step-1: Calculate turnout in each county
county_president_votes = df2024_president_county[['state_code', 'county_name', 'total_votes']]
merged = pd.merge(
    df_county_details,
    county_president_votes,
    on=['state_code', 'county_name'],
    how='outer',
    indicator=True
)
# merged.to_csv('FinalDatasets/interim/mismatches1.csv', index=False, encoding='utf-8-sig')
merged['turnout'] = merged['total_votes'] / merged['population']
df_county_details = merged.loc[merged['_merge'] == 'both', 
                ['state_name', 'state_code', 'county_name', 'population', 'total_votes', 'turnout']].reset_index(drop=True)
print(df_county_details.columns)

# df_county_details.to_csv('FinalDatasets/interim/county-details-final-v1.csv', index=False, encoding='utf-8-sig')

# Step-2: Add education attainment details for each county
df_county_education = pd.read_csv('Datasets/counties-education.csv', encoding=result['encoding'])
df_county_education.drop(index=df_county_education.index[0], axis=0, inplace=True)
df_county_education = df_county_education[['NAME','college_educated']]
df_county_education.rename(columns={'Total-CollegeEducated': 'college_educated'}, inplace=True) 
df_county_education['state_name'] = df_county_education['NAME'].str.split(', ').str[1]
df_county_education["state_code"] = df_county_education["state_name"].map(us_state_to_code)
df_county_education["county_name"] = df_county_education['NAME'].str.split(', ').str[0]
df_county_education = df_county_education[['state_name', 'state_code', 'county_name', 'college_educated']]

merged = pd.merge(
    df_county_details,
    df_county_education,
    on=['state_name', 'state_code', 'county_name'],
    how='outer',        # keep everything so we can spot non-matches
    indicator=True      # adds a "_merge" column
)
# merged.to_csv('FinalDatasets/interim/mismatches2.csv', index=False, encoding='utf-8-sig')
df_county_details = merged.loc[merged['_merge'] == 'both', 
                ['state_name', 'state_code', 'county_name', 'population', 'total_votes', 'turnout', 'college_educated']].reset_index(drop=True)
# mismatches = merged.loc[merged['_merge'] != 'both', ['state_code', 'county_name', '_merge']].reset_index(drop=True)
# df_county_details.to_csv('FinalDatasets/interim/county-details-final-v2.csv', index=False, encoding='utf-8-sig')
# print(df_county_details.columns)

# Step-3: Add income details for each county
df_county_income = pd.read_csv('Datasets/counties-income.csv', encoding=result['encoding'])
df_county_income.drop(index=df_county_income.index[0], axis=0, inplace=True)
df_county_income = df_county_income[['NAME','households_total', 'households_mean_income', 'households_median_income', 'families_total', 'families_mean_income', 'families_median_income']]
df_county_income['state_name'] = df_county_income['NAME'].str.split(', ').str[1]
df_county_income["state_code"] = df_county_income["state_name"].map(us_state_to_code)
df_county_income["county_name"] = df_county_income['NAME'].str.split(', ').str[0]
df_county_income = df_county_income[['state_name', 'state_code', 'county_name', 'households_total', 'households_mean_income', 'households_median_income', 'families_total', 'families_mean_income', 'families_median_income']]

merged = pd.merge(
    df_county_details,
    df_county_income,
    on=['state_name', 'state_code', 'county_name'],
    how='outer',        # keep everything so we can spot non-matches
    indicator=True      # adds a "_merge" column
)

df_county_details = merged.loc[merged['_merge'] == 'both', 
                ['state_name', 'state_code', 'county_name', 'population', 'total_votes', 'turnout', 'college_educated', 'households_total', 'households_mean_income', 'households_median_income', 'families_total', 'families_mean_income', 'families_median_income']].reset_index(drop=True)
# mismatches = merged.loc[merged['_merge'] != 'both', ['state_code', 'county_name', '_merge']].reset_index(drop=True)
# df_county_details.to_csv('FinalDatasets/interim/county-details-final-v3.csv', index=False, encoding='utf-8-sig')

# Step-4: Add poverty details
df_county_poverty = pd.read_csv('Datasets/counties-poverty.csv', encoding=result['encoding'])
df_county_poverty.drop(index=df_county_poverty.index[0], axis=0, inplace=True)
df_county_poverty = df_county_poverty[['NAME', 'poverty_total', 'poverty_percent']]
df_county_poverty['state_name'] = df_county_poverty['NAME'].str.split(', ').str[1]
df_county_poverty["state_code"] = df_county_poverty["state_name"].map(us_state_to_code)
df_county_poverty["county_name"] = df_county_poverty['NAME'].str.split(', ').str[0]
df_county_poverty = df_county_poverty[['state_name', 'state_code', 'county_name', 'poverty_total', 'poverty_percent']]

merged = pd.merge(
    df_county_details,
    df_county_poverty,
    on=['state_name', 'state_code', 'county_name'],
    how='outer',
    indicator=True
)
# merged.to_csv('FinalDatasets/interim/mismatches4.csv', index=False, encoding='utf-8-sig')
df_county_details = merged.loc[merged['_merge'] == 'both', 
                ['state_name', 'state_code', 'county_name', 'population', 'total_votes', 'turnout', 'college_educated', 'households_total', 'households_mean_income', 'households_median_income', 'families_total', 'families_mean_income', 'families_median_income', 'poverty_total', 'poverty_percent']].reset_index(drop=True)

merged = pd.merge(
    df_county_details,
    df2024_president_county[['state_name', 'state_code', 'county_name', 'per_point_diff']],
    on=['state_name', 'state_code', 'county_name'],
    how='outer',
    indicator=True
)
# merged.to_csv('FinalDatasets/interim/mismatches5.csv', index=False, encoding='utf-8-sig')
df_county_details = merged.loc[merged['_merge'] == 'both', 
                ['state_name', 'state_code', 'county_name', 'population', 'total_votes', 'turnout', 'college_educated', 'households_total', 'households_mean_income', 'households_median_income', 'families_total', 'families_mean_income', 'families_median_income', 'poverty_total', 'poverty_percent', 'per_point_diff']].reset_index(drop=True)
df_county_details.rename(columns={'per_point_diff':'diff_percent'}, inplace=True)

numeric_cols = [
    "population", "total_votes", "turnout", "college_educated",
    "households_total", "households_mean_income", "households_median_income",
    "families_total", "families_mean_income", "families_median_income",
    "poverty_total", "poverty_percent"
]

for col in numeric_cols:
    # coerce the column to numeric, bad parses become NaN
    coerced = pd.to_numeric(df_county_details[col], errors="coerce")
    # find the rows where the original was non-null but coercion gave NaN
    bad_idx = df_county_details.index[
        coerced.isna() & df_county_details[col].notna()
    ].tolist()
    if bad_idx:
        print(f"\nColumn `{col}` has {len(bad_idx)} invalid entries at rows: {bad_idx}")
        print("   Example values:", df_county_details.loc[bad_idx, col].unique())
df_county_details[numeric_cols] = df_county_details[numeric_cols].astype(float)

df_county_details['poverty_percent'] = (df_county_details['poverty_total']/df_county_details['population']) * 100

# Add percentage of college educated people
df_county_details['college_educated_percent'] = (df_county_details['college_educated']/df_county_details['population']) * 100
df_county_details['turnout'] = df_county_details['turnout']*100
df_county_details.loc[df_county_details['turnout'] > 100, 'turnout'] = 100
df_county_details.rename(columns={'turnout':'turnout_percent'}, inplace=True)
df_county_details.to_csv('FinalDatasets/2024-county-details-final.csv', index=False, encoding='utf-8-sig')

Index(['state_name', 'state_code', 'county_name', 'population', 'total_votes',
       'turnout'],
      dtype='object')


In [36]:
# Part-7: Extract presidential results by congressional district
df2024_president_cd_results = pd.read_csv('Datasets/2024-results-cdlevel.csv', encoding=result['encoding'])

cols = ['district', 'd_total', 'r_total', 'other_total', 'total_votes', 'd_percent', 'r_percent', 'other_percent','margin']
df2024_president_cd_results = df2024_president_cd_results[cols].copy()
# numeric_cols = ['d_total', 'r_total', 'other_total', 'total_votes', 'd_percent', 'r_percent', 'other_percent','margin']
# df2024_president_cd_results[numeric_cols] = df2024_president_cd_results[numeric_cols].apply(pd.to_numeric, errors='coerce')

for c in df2024_president_cd_results.columns.drop('district'):
    df2024_president_cd_results[c] = df2024_president_cd_results[c].astype(str).str.replace(r'[%,]', '', regex=True)
    df2024_president_cd_results[c] = df2024_president_cd_results[c].astype(float)

df2024_president_cd_results['district'] = (
    df2024_president_cd_results['district']
      .str.replace('-', ' ', regex=False)
      .str.replace(r'\b AL\b', ' 01', regex=True)
)

df2024_president_cd_results['margin'] = -df2024_president_cd_results['margin']

df2024_president_cd_results.to_csv('FinalDatasets/2024-president-cdlevel-results-final.csv', index=False, encoding='utf-8-sig')

In [37]:
# Part-8: Clean state level details
df_state_details = pd.read_csv('Datasets/2024-state-details.csv', encoding=result['encoding'])
df_state_details["state_code"] = df_state_details["state_name"].map(us_state_to_code)

# Step 1: Remove ',', '$', and '%' from all string values
df_state_details = df_state_details.replace({',': '', '\$': '', '%': ''}, regex=True)

# Step 2: Convert all columns except 'state_name', 'state_code', 'fips_code' to float
cols_to_convert = df_state_details.columns.difference(['state_name', 'state_code', 'fips_code'])
df_state_details[cols_to_convert] = df_state_details[cols_to_convert].astype(float).round(2)

# Step-3: Add state victory margin
merged = pd.merge(
    df_state_details,
    df2024_president_state[['state_name', 'state_code', 'diff_percent']],
    on=['state_name', 'state_code'],
    how='outer',
    indicator=True
)
# merged.to_csv('FinalDatasets/interim/mismatches6.csv', index=False, encoding='utf-8-sig')
df_state_details = merged.loc[merged['_merge'] == 'both', 
    [col for col in merged.columns if col != '_merge']
].reset_index(drop=True)

# Srep-4: Add turnout details
merged = pd.merge(
    df_state_details,
    df2024_turnout_president[['state_name', 'state_code', 'vep_turnout_rate']],
    on=['state_name', 'state_code'],
    how='outer',
    indicator=True
)
# merged.to_csv('FinalDatasets/interim/mismatches-turnoutstatedetails.csv', index=False, encoding='utf-8-sig')
df_state_details = merged.loc[merged['_merge'] == 'both', 
    [col for col in merged.columns if col != '_merge']
].reset_index(drop=True)

df_state_details.rename(columns={'vep_turnout_rate':'turnout_percent'}, inplace=True)
df_state_details['turnout_percent'] = df_state_details['turnout_percent'].str.replace(',', '').str.replace('%', '').astype(float)
df_state_details['other_percent'] = 100 - df_state_details['white_percent'] - df_state_details['black_percent'] - df_state_details['native_percent'] - df_state_details['asian_percent'] - df_state_details['pacific_percent']
df_state_details.to_csv('FinalDatasets/2024-state-details-final.csv', index=False, encoding='utf-8-sig')

<>:6: SyntaxWarning: invalid escape sequence '\$'
<>:6: SyntaxWarning: invalid escape sequence '\$'
C:\Users\prana\AppData\Local\Temp\ipykernel_10200\1420176813.py:6: SyntaxWarning: invalid escape sequence '\$'
  df_state_details = df_state_details.replace({',': '', '\$': '', '%': ''}, regex=True)


In [38]:
# Part-9: County Race Details
df2024_county_race_details = pd.read_csv('Datasets/2024-county-race-details.csv', encoding=result['encoding'])
df2024_county_race_details["state_code"] = df2024_county_race_details["state_name"].map(us_state_to_code)

# Include victory margin
merged = pd.merge(
    df2024_county_race_details,
    df_county_details[['state_name', 'state_code', 'county_name', 'diff_percent', 'turnout_percent']],
    on=['state_name', 'state_code', 'county_name'],
    how='outer',
    indicator=True
)
# merged.to_csv('FinalDatasets/interim/mismatches-countyracedetails.csv', index=False, encoding='utf-8-sig')
df2024_county_race_details = merged.loc[merged['_merge'] == 'both', 
    [col for col in merged.columns if col != '_merge']
].reset_index(drop=True)

# Initialize County Details dataframe
numeric_cols = [
    'population', 'white_percent', 'black_percent', 'native_percent',
    'asian_percent', 'pacific_percent', 'other_percent', 'hispanic_percent', 'diff_percent', 'turnout_percent'
]
df2024_county_race_details[numeric_cols] = df2024_county_race_details[numeric_cols].astype(float)
df2024_county_race_details = df2024_county_race_details[['state_name', 'state_code', 'county_name'] + numeric_cols]
df2024_county_race_details.to_csv('FinalDatasets/2024-county-race-details-final.csv', index=False, encoding='utf-8-sig')